# ADS 509 Module 1: APIs and Web Scraping

This notebook has three parts. In the first part you will pull data from the Twitter API. In the second, you will scrape lyrics from AZLyrics.com. In the last part, you'll run code that verifies the completeness of your data pull. 

For this assignment you have chosen two musical artists who have at least 100,000 Twitter followers and 20 songs with lyrics on AZLyrics.com. In this part of the assignment we pull the some of the user information for the followers of your artist and store them in text files. 


## Important Note

This assignment requires you to have a version of Tweepy that is at least version 4. The latest version is 4.10 as I write this. Critically, this version of Tweepy is *not* on the upgrade path from Version 3, so you will not be able to simply upgrade the package if you are on Version 3. Instead you will need to explicitly install version 4, which you can do with a command like this: `pip install "tweepy>=4"`. You will also be using Version 2 of the Twitter API for this assignment. 

Run the below cell. If your version of Tweepy begins with a "4", then you should be good to go. If it begins with a "3" then run the following command, found [here](https://stackoverflow.com/questions/5226311/installing-specific-package-version-with-pip), at the command line or in a cell: `pip install -Iv tweepy==4.9`. (You may want to update that version number if Tweepy has moved on past 4.9. 

In [1]:
pip show tweepy

Name: tweepy
Version: 4.10.1
Summary: Twitter library for Python
Home-page: https://www.tweepy.org/
Author: Joshua Roesslein
Author-email: tweepy@googlegroups.com
License: MIT
Location: c:\users\drzag\anaconda3\lib\site-packages
Requires: requests, oauthlib, requests-oauthlib
Required-by: 
Note: you may need to restart the kernel to use updated packages.


## General Assignment Instructions

These instructions are included in every assignment, to remind you of the coding standards for the class. Feel free to delete this cell after reading it. 

One sign of mature code is conforming to a style guide. We recommend the [Google Python Style Guide](https://google.github.io/styleguide/pyguide.html). If you use a different style guide, please include a cell with a link. 

Your code should be relatively easy-to-read, sensibly commented, and clean. Writing code is a messy process, so please be sure to edit your final submission. Remove any cells that are not needed or parts of cells that contain unnecessary code. Remove inessential `import` statements and make sure that all such statements are moved into the designated cell. 

Make use of non-code cells for written commentary. These cells should be grammatical and clearly written. In some of these cells you will have questions to answer. The questions will be marked by a "Q:" and will have a corresponding "A:" spot for you. *Make sure to answer every question marked with a `Q:` for full credit.* 


# Twitter API Pull

In [143]:
# for the twitter section
import tweepy
import os
import datetime
import re
from pprint import pprint

# for the lyrics scrape section
import requests
import time
from bs4 import BeautifulSoup
from collections import defaultdict, Counter


In [27]:
#from google.colab import drive
#drive.mount('/content/drive')

In [144]:
import pandas as pd
import numpy as np
import random

We need bring in our API keys. Since API keys should be kept secret, we'll keep them in a file called `api_keys.py`. This file should be stored in the directory where you store this notebook. The example file is provided for you on Blackboard. The example has API keys that are _not_ functional, so you'll need to get Twitter credentials and replace the placeholder keys. 

CODE CHANGE FOR COLAB: The next code cell needed the path defined before the notebook could recognize the apikeys.py file as a module

In [31]:
import sys
#The next line adds a path for apikeys.py in Colab
#sys.path.insert(0,os.path.abspath('/content/drive/MyDrive/Colab Notebooks'))

import apikeys
from apikeys import api_key,api_key_secret,bearer_token

###Had to hardcode the bearer token to get this to work in Python

In [4]:
#client = tweepy.Client(bearer_token,wait_on_rate_limit=True)
client = tweepy.Client('AAAAAAAAAAAAAAAAAAAAADGDgwEAAAAAt1VEWj5vUNphL%2BQoRizQXZjfiyY%3DxfAh1oJYepPoNZN5gX7uRR2AxwbwTLGr5SMM420TX1VpQX5yPP',wait_on_rate_limit=True)
client

# Testing the API

The Twitter APIs are quite rich. Let's play around with some of the features before we dive into this section of the assignment. For our testing, it's convenient to have a small data set to play with. We will seed the code with the handle of John Chandler, one of the instructors in this course. His handle is `@37chandler`. Feel free to use a different handle if you would like to look at someone else's data. 

We will write code to explore a few aspects of the API: 

1. Pull some of the followers @37chandler.
1. Explore response data, which gives us information about Twitter users. 
1. Pull the last few tweets by @37chandler.


In [33]:
handle = "37chandler"
user_obj = client.get_user(username=handle, user_fields=["public_metrics"])

followers = client.get_users_followers(
    # Learn about user fields here: 
    # https://developer.twitter.com/en/docs/twitter-api/data-dictionary/object-model/user
    user_obj.data.id, user_fields=["created_at","description","location",
                                   "public_metrics"]
)

#To be cleaned up
print(user_obj)
print(user_obj.data.id)
print(user_obj.data.name)
print(user_obj.data.username)
print(user_obj.data.public_metrics)
followers.data


Response(data=<User id=33029025 name=John Chandler username=37chandler>, includes={}, errors=[], meta={})
33029025
John Chandler
37chandler
{'followers_count': 193, 'following_count': 589, 'tweet_count': 995, 'listed_count': 3}


[<User id=1553891535040630785 name=Dave Renn username=CocoDibonet>,
 <User id=618734476 name=Lionel username=LSW1138>,
 <User id=401835058 name=Megan Randall username=sugarfriss>,
 <User id=634650371 name=Jacob Salzman username=bakedjakebakes>,
 <User id=1227344433622003712 name=twiter not fun username=AmreinCurtis>,
 <User id=2989225576 name=Hariettwilsonincarnate username=angel28117026>,
 <User id=1478401670274957312 name=Christian Tinsley username=ChristianTinsl8>,
 <User id=257285645 name=Steve username=HicSvntDraconez>,
 <User id=1469785454576820225 name=John O'Connor 🇺🇦 username=JohnOCo70713197>,
 <User id=1181131341687066624 name=CodeGrade username=CodeGradeCom>,
 <User id=257686741 name=Cleverhood username=cleverhoods>,
 <User id=80746014 name=Regina 🚶‍♀️🚲🌳 username=mplsFietser>,
 <User id=174368082 name=Eric Hallstrom username=echallstrom>,
 <User id=824723466196770817 name=Tyler 📊 🐕 🚲 username=byler_t117>,
 <User id=975810358777544705 name=The Center for Community Ownership (

Now let's explore these a bit. We'll start by printing out names, locations, following count, and followers count for these users. 

In [34]:
num_to_print = 20

for idx, user in enumerate(followers.data) :
    following_count = user.public_metrics['following_count']
    followers_count = user.public_metrics['followers_count']
    
    print(f"{user.name} lists '{user.location}' as their location.")
    print(f" Following: {following_count}, Followers: {followers_count}.")
    print()
    
    if idx >= (num_to_print - 1) :
        break


Dave Renn lists 'None' as their location.
 Following: 42, Followers: 10.

Lionel lists 'None' as their location.
 Following: 202, Followers: 204.

Megan Randall lists 'None' as their location.
 Following: 141, Followers: 100.

Jacob Salzman lists 'None' as their location.
 Following: 562, Followers: 134.

twiter not fun lists 'None' as their location.
 Following: 222, Followers: 21.

Hariettwilsonincarnate lists 'None' as their location.
 Following: 218, Followers: 60.

Christian Tinsley lists 'None' as their location.
 Following: 2, Followers: 0.

Steve lists 'I'm over here.' as their location.
 Following: 1590, Followers: 33.

John O'Connor 🇺🇦 lists 'None' as their location.
 Following: 8, Followers: 1.

CodeGrade lists 'Amsterdam' as their location.
 Following: 2820, Followers: 421.

Cleverhood lists 'Providence, RI' as their location.
 Following: 2792, Followers: 3558.

Regina 🚶‍♀️🚲🌳 lists 'Minneapolis' as their location.
 Following: 2796, Followers: 3322.

Eric Hallstrom lists 'Mi

Let's find the person who follows this handle who has the most followers. 

In [35]:
max_followers = 0

for idx, user in enumerate(followers.data) :
    followers_count = user.public_metrics['followers_count']
    
    if followers_count > max_followers :
        max_followers = followers_count
        max_follower_user = user

        
print(max_follower_user)
print(max_follower_user.public_metrics)


WedgeLIVE
{'followers_count': 14165, 'following_count': 2223, 'tweet_count': 56084, 'listed_count': 218}


Let's pull some more user fields and take a look at them. The fields can be specified in the `user_fields` argument. 

In [37]:
response = client.get_user(id=user_obj.data.id,
                          user_fields=["created_at","description","location",
                                       "entities","name","pinned_tweet_id","profile_image_url",
                                       "verified","public_metrics"])

In [38]:
for field, value in response.data.items() :
    print(f"for {field} we have {value}")


for created_at we have 2009-04-18 22:08:22+00:00
for name we have John Chandler
for username we have 37chandler
for description we have He/Him. Data scientist, urban cyclist, educator, erstwhile frisbee player. 

¯\_(ツ)_/¯
for id we have 33029025
for verified we have False
for public_metrics we have {'followers_count': 193, 'following_count': 589, 'tweet_count': 995, 'listed_count': 3}
for location we have MN
for profile_image_url we have https://pbs.twimg.com/profile_images/2680483898/b30ae76f909352dbae5e371fb1c27454_normal.png


Now a few questions for you about the user object.

Q: How many fields are being returned in this user object? 

A: 9 <!-- Put your answer here --> 

---

Q: Are any of the fields within the user object non-scalar? (I.e., more complicated than a simple data type like integer, float, string, boolean, etc.) 

A: public_metrics has a dict-like structure <!-- Put your answer here --> 

---

Q: How many friends, followers, and tweets does this user have? 

A: Following 589, Followers is 193, tweet count is 995<!-- Put your answer here --> 


Although you won't need it for this assignment, individual tweets can be a rich source of text-based data. To illustrate the concepts, let's look at the last few tweets for this user. You are encouraged to explore the fields that are available about Tweets.

In [39]:
response = client.get_users_tweets(user_obj.data.id)

# By default, only the ID and text fields of each Tweet will be returned
for idx, tweet in enumerate(response.data) :
    print(tweet.id)
    print(tweet.text)
    print()
    
    if idx > 10 :
        break

1568982292923826176
RT @wonderofscience: Amazing lenticular cloud over Mount Fuji

Credit: Iurie Belegurschi
https://t.co/0mUxl28H9U

1568242374085869570
RT @depthsofwiki: lots of memes about speedy wikipedia editors — quick thread about what went down on wikipedia in the minutes after her de…

1568074978754703361
@DrLaurenWilson @leighradwood @MaritsaGeorgiou @Walgreens I could not possibly agree more with this sentiment. Compared to almost any other primary care I've received, they are great.

1567530169686196224
@DrLaurenWilson @MaritsaGeorgiou @Walgreens For those who have access to Curry Health Center on campus, you can get a bivalent booster in 15 minutes from their delightful staff.

1567511181526708224
RT @shes_the_maNN1: I can’t describe how ancient this makes me feel. https://t.co/a1IvELjOFY

1567510612665864193
RT @AngryBlackLady: this is hilarious

1566031636457725953
RT @MarkJacob16: With all the arguments over whether MAGA Republicans are fascists, I reread William Shirer

## Pulling Follower Information

In this next section of the assignment, we will pull information about the followers of your two artists. We've seen above how to pull a set of followers using `client.get_users_followers`. This function has a parameter, `max_results`, that we can use to change the number of followers that we pull. Unfortunately, we can only pull 1000 followers at a time, which means we will need to handle the _pagination_ of our results. 

The return object has the `.data` field, where the results will be found. It also has `.meta`, which we use to select the next "page" in the results using the `next_token` result. I will illustrate the ideas using our user from above. 


### Rate Limiting

Twitter limits the rates at which we can pull data, as detailed in [this guide](https://developer.twitter.com/en/docs/twitter-api/rate-limits). We can make 15 user requests per 15 minutes, meaning that we can pull $4 \cdot 15 \cdot 1000 = 60000$ users per hour. I illustrate the handling of rate limiting below, though whether or not you hit that part of the code depends on your value of `handle`.  


In the below example, I'll pull all the followers, 25 at a time. (We're using 25 to illustrate the idea; when you do this set the value to 1000.) 

TODO: Get artist counts and estimate total time.

In [182]:
#  THE FOLLOWING CODE HAS BEEN INCUDED AND MODIFIED IN THE FULL CODE BLOCK BELOW
'''
handle_followers = []
pulls = 0
max_pulls = 3
next_token = None

while True :

    followers = client.get_users_followers(
        user_obj.data.id, 
        max_results=1, # when you do this for real, set this to 1000!
        pagination_token = next_token,
        user_fields=["created_at","description","location",
                     "entities","name","pinned_tweet_id","profile_image_url",
                     "verified","public_metrics"]
    )
    pulls += 1
    
    for follower in followers.data : 
        follower_row = (follower.id,follower.name,follower.created_at,follower.description)
        handle_followers.append(follower_row)
    
    if 'next_token' in followers.meta and pulls < max_pulls :
        next_token = followers.meta['next_token']
        print(next_token)
    else : 
        break

handle_followers
'''

'\nhandle_followers = []\npulls = 0\nmax_pulls = 3\nnext_token = None\n\nwhile True :\n\n    followers = client.get_users_followers(\n        user_obj.data.id, \n        max_results=1, # when you do this for real, set this to 1000!\n        pagination_token = next_token,\n        user_fields=["created_at","description","location",\n                     "entities","name","pinned_tweet_id","profile_image_url",\n                     "verified","public_metrics"]\n    )\n    pulls += 1\n    \n    for follower in followers.data : \n        follower_row = (follower.id,follower.name,follower.created_at,follower.description)\n        handle_followers.append(follower_row)\n    \n    if \'next_token\' in followers.meta and pulls < max_pulls :\n        next_token = followers.meta[\'next_token\']\n        print(next_token)\n    else : \n        break\n\nhandle_followers\n'

Now let's take a look at your artists and see how long it is going to take to pull all their followers. 

In [41]:
handle = "didoofficial"
user_obj = client.get_user(username=handle, user_fields=["public_metrics"])
print(user_obj.data.public_metrics['followers_count'])

117585


In [42]:
handle = "aboveandbeyond"
user_obj = client.get_user(username=handle, user_fields=["public_metrics"])
print(user_obj.data.public_metrics['followers_count'])

541728


In [ ]:
#artists['cher'].public_metrics

In [43]:
artists = dict()

for handle in ['didoofficial','aboveandbeyond'] : 
    user_obj = client.get_user(username=handle,user_fields=["public_metrics"])
    artists[handle] = (user_obj.data.id, 
                       handle,
                       user_obj.data.public_metrics['followers_count'])
    

for artist, data in artists.items() : 
    print(f"It would take {data[2]/(1000*15*4):.2f} hours to pull all {data[2]} followers for {artist}. ")
    

It would take 1.96 hours to pull all 117585 followers for didoofficial. 
It would take 9.03 hours to pull all 541728 followers for aboveandbeyond. 


Depending on what you see in the display above, you may want to limit how many followers you pull. It'd be great to get at least 200,000 per artist. 

As we pull data for each artist we will write their data to a folder called "twitter", so we will make that folder if needed.

In [181]:
# Make the "twitter" folder here. If you'd like to practice your programming, add functionality 
# that checks to see if the folder exists. If it does, then "unlink" it. Then create a new one.

if os.path.isdir("twitter") : 
    shutil.rmtree("twitter/")

os.mkdir("twitter")

In this following cells, build on the above code to pull some of the followers and their data for your two artists. As you pull the data, write the follower ids to a file called `[artist name]_followers.txt` in the "twitter" folder. For instance, for Cher I would create a file named `cher_followers.txt`. As you pull the data, also store it in an object like a list or a data frame.

Extract and store the following fields: 

* screen_name	
* name	
* id	
* location	
* followers_count	
* friends_count	
* description

 Store the fields with one user per row in a tab-delimited text file with the name `[artist name]_follower_data.txt`. For instance, for Cher I would create a file named `cher_follower_data.txt`. 


One note: the user's description can have tabs or returns in it, so make sure to clean those out of the description before writing them to the file. I've included some example code to do that below the stub. 

In [ ]:
# num_followers_to_pull = 200*1000 # feel free to use this to limit the number of followers you pull.

In [45]:
# Modify the below code stub to pull the follower IDs and write them to a file. 
# Grabs the time when we start making requests to the API
start_time = datetime.datetime.now()

# Pull the follower IDs

handles = ['didoofficial','aboveandbeyond'] 
for handle in handles :
    handle_followers = []
    pulls = 0
    max_pulls = 2
    next_token = None
    user_obj = client.get_user(username=handle)
    output_file = handle + "_followers.txt"
     
    while True :
        followers = client.get_users_followers(
            user_obj.data.id,
            max_results=3, # when you do this for real, set this to 1000!
            pagination_token = next_token,
            user_fields=["created_at","description","location",
                         "entities","name","pinned_tweet_id","profile_image_url",
                         "verified","public_metrics","username"]
                         )
        print(pulls+1, "pulls completed for", handle)
        print('Sleeping for about 60 seconds')
        time.sleep(130*random.random())
        pulls += 1


# Extract the information you need for the followers        
        for follower in followers.data :
            follower_row = (follower.name,
                            follower.username,
                            follower.id,
                            follower.location,
                            follower.created_at,
                            follower.public_metrics['followers_count'],
                            follower.public_metrics['following_count'],
                            (re.sub(r"\s+"," ",follower.description))
                            )
            handle_followers.append(follower_row)

        handle_followers_df = pd.DataFrame (handle_followers, columns = ['name', 'username','id','location','created',
                                                        'followers','following','description'])
        
        if 'next_token' in followers.meta and pulls < max_pulls :
            next_token = followers.meta['next_token']
            print(next_token)
        else :
            break
        
# Write the data to the output file in the `twitter` folder.
        
    handle_followers_df.to_csv(output_file,
                               index=False,
                               header=False,
                               sep='\t')

# If you've pulled num_followers_to_pull, feel free to break out paged twitter API response
            
# Let's see how long it took to grab all follower IDs
end_time = datetime.datetime.now()
print(end_time - start_time)

1 pulls completed for didoofficial
Sleeping for about 60 seconds
81P1775KLGP1GZZZ
2 pulls completed for didoofficial
Sleeping for about 60 seconds
1 pulls completed for aboveandbeyond
Sleeping for about 60 seconds
N100M34UU0P1GZZZ
2 pulls completed for aboveandbeyond
Sleeping for about 60 seconds
0:02:45.438087


In [ ]:
tricky_description = """
    Home by Warsan Shire
    
    no one leaves home unless
    home is the mouth of a shark.
    you only run for the border
    when you see the whole city
    running as well.

"""
# This won't work in a tab-delimited text file.

clean_description = re.sub(r"\s+"," ",tricky_description)
clean_description

' Home by Warsan Shire no one leaves home unless home is the mouth of a shark. you only run for the border when you see the whole city running as well. '

---

# Lyrics Scrape

This section asks you to pull data from the Twitter API and scrape www.AZLyrics.com. In the notebooks where you do that work you are asked to store the data in specific ways. 

In [145]:
import requests
from bs4 import BeautifulSoup
import lxml
from urllib.request import Request, urlopen
from tqdm import tqdm
import json
import time

In [146]:
artists = {'dido':"https://www.azlyrics.com/d/dido.html",
           'abovebeyond':"https://www.azlyrics.com/a/abovebeyond.html"} 

# we'll use this dictionary to hold both the artist name and the link on AZlyrics

#Clean up: Assign Wikipedia page as URL
#artists = {'didoofficial':"https://en.wikipedia.org/wiki/55_Wall_Street.html"} 

## A Note on Rate Limiting

The lyrics site, www.azlyrics.com, does not have an explicit maximum on number of requests in any one time, but in our testing it appears that too many requests in too short a time will cause the site to stop returning lyrics pages. (Entertainingly, the page that gets returned seems to only have the song title to [a Tom Jones song](https://www.azlyrics.com/lyrics/tomjones/itsnotunusual.html).) 

Whenever you call `requests.get` to retrieve a page, put a `time.sleep(5 + 10*random.random())` on the next line. This will help you not to get blocked. If you _do_ get blocked, which you can identify if the returned pages are not correct, just request a lyrics page through your browser. You'll be asked to perform a CAPTCHA and then your requests should start working again. 

## Part 1: Finding Links to Songs Lyrics

That general artist page has a list of all songs for that artist with links to the individual song pages. 

Q: Take a look at the `robots.txt` page on www.azlyrics.com. (You can read more about these pages [here](https://developers.google.com/search/docs/advanced/robots/intro).) Is the scraping we are about to do allowed or disallowed by this page? How do you know? 

A: According to their robot.txt file, allowed for all crawlers (except 008, 80legs) for all pages except /songs/ and /lyricsdb/.


In [147]:
# Let's set up a dictionary of lists to hold our links
lyrics_pages = defaultdict(list)

for artist, artist_page in artists.items() :
    # request the page and sleep
    r = requests.get(artist_page)
    time.sleep(5 + 10*random.random())
    
    # now extract the links to lyrics pages from this page
    urls = []
    soup = BeautifulSoup(r.content, 'lxml') # text, html.parser
    for link in soup.findAll('a'):  #find_all
        if ("lyrics/" + artist) in str(link.get('href')):
            urls.append('https://www.azlyrics.com' + str(link.get('href')))

    lyrics_pages[artist] = urls

    # store the links `lyrics_pages` where the key is the artist and the
    # value is a list of links. 

print(lyrics_pages)

defaultdict(<class 'list'>, {'dido': ['https://www.azlyrics.com/lyrics/dido/allyouwant.html', 'https://www.azlyrics.com/lyrics/dido/dontthinkofme.html', 'https://www.azlyrics.com/lyrics/dido/herewithme.html', 'https://www.azlyrics.com/lyrics/dido/honestlyok.html', 'https://www.azlyrics.com/lyrics/dido/hunter.html', 'https://www.azlyrics.com/lyrics/dido/imnoangel.html', 'https://www.azlyrics.com/lyrics/dido/isobel.html', 'https://www.azlyrics.com/lyrics/dido/mylife.html', 'https://www.azlyrics.com/lyrics/dido/myloversgone.html', 'https://www.azlyrics.com/lyrics/dido/slide.html', 'https://www.azlyrics.com/lyrics/dido/takemyhand.html', 'https://www.azlyrics.com/lyrics/dido/thankyou.html', 'https://www.azlyrics.com/lyrics/dido/whiteflag.html', 'https://www.azlyrics.com/lyrics/dido/stoned.html', 'https://www.azlyrics.com/lyrics/dido/lifeforrent.html', 'https://www.azlyrics.com/lyrics/dido/marysinindia.html', 'https://www.azlyrics.com/lyrics/dido/seeyouwhenyoure40.html', 'https://www.azlyric

Let's make sure we have enough lyrics pages to scrape. 

In [148]:
for artist, lp in lyrics_pages.items() :
    assert(len(set(lp)) > 20) 

In [149]:
# Let's see how long it's going to take to pull these lyrics 
# if we're waiting `5 + 10*random.random()` seconds 
for artist, links in lyrics_pages.items() : 
    print(f"For {artist} we have {len(links)}.")
    print(f"The full pull will take for this artist will take {round(len(links)*10/3600,2)} hours.")

For dido we have 83.
The full pull will take for this artist will take 0.23 hours.
For abovebeyond we have 71.
The full pull will take for this artist will take 0.2 hours.


## Part 2: Pulling Lyrics

Now that we have the links to our lyrics pages, let's go scrape them! Here are the steps for this part. 

1. Create an empty folder in our repo called "lyrics". 
1. Iterate over the artists in `lyrics_pages`. 
1. Create a subfolder in lyrics with the artist's name. For instance, if the artist was Cher you'd have `lyrics/cher/` in your repo.
1. Iterate over the pages. 
1. Request the page and extract the lyrics from the returned HTML file using BeautifulSoup.
1. Use the function below, `generate_filename_from_url`, to create a filename based on the lyrics page, then write the lyrics to a text file with that name. 


In [150]:
import shutil

In [151]:
# Make the lyrics folder here. If you'd like to practice your programming, add functionality 
# that checks to see if the folder exists. If it does, then use shutil.rmtree to remove it and create a new one.

if os.path.isdir("lyrics") : 
    shutil.rmtree("lyrics/")

os.mkdir("lyrics")
os.mkdir("lyrics/dido")
os.mkdir("lyrics/abovebeyond")

In [154]:
url_stub = "https://www.azlyrics.com" 
start_time = datetime.datetime.now()

def generate_filename_from_link(link) :
    
    if not link :
        return None
    
    # drop the http or https and the html

    name = link.replace("www.azlyrics.com","")
    name = name.replace("https","")
    name = name.replace("http","")
    name = name.replace(".html","")
    name = name.replace("/lyrics/","")
    
    # Replace useless chareacters with UNDERSCORE
    name = name.replace("://","").replace(".","_").replace("/","_")
    
    # tack on .txt
    name = name + ".txt"
    
    return(name)

total_pages = 0 

for artist, links in lyrics_pages.items():
    if artist == 'dido':
        save_path = 'C:/Users/drzag/lyrics/dido/'
    if artist == 'abovebeyond':
        save_path = 'C:/Users/drzag/lyrics/abovebeyond/'
    if(isinstance(links, list)):
        for link in links:
            
            lyric_list = []
            r = requests.get(link)
            time.sleep(5 + 10*random.random())
            soup = BeautifulSoup(r.text, 'lxml')
            
            
            lyrics = []
            flag = False
            for string in soup.body.stripped_strings:
                if string == 'Submit Corrections':
                    break
                if flag==True:
                    lyrics.append(string)
                if string == 'Search':
                    flag=True
  
            print(lyrics)
    
            output_file = generate_filename_from_link(link)
            
            complete_name = os.path.join(save_path, output_file)
            with open(complete_name, 'w', encoding = "utf-8") as f:
                for line in lyrics:
                    f.write(line+'\n')
            f.close()
    

 
end_time = datetime.datetime.now()

print(end_time - start_time)

            #request the lyrics pages
            #Sleep
            #Extract the title and lyrics from the page
            #Write out the title, two returns ('\n'), and the lyrics. Use `generate_filename_from_url`
            
#for artist in lyrics_pages :

    # Use this space to carry out the following steps: 
    
    # 1. Build a subfolder for the artist
    # 2. Iterate over the lyrics pages
    # 3. Request the lyrics page. 
        # Don't forget to add a line like `time.sleep(5 + 10*random.random())`
        # to sleep after making the request
    # 4. Extract the title and lyrics from the page.
    # 5. Write out the title, two returns ('\n'), and the lyrics. Use `generate_filename_from_url`
    #    to generate the filename. 
    
    # Remember to pull at least 20 songs per artist. It may be fun to pull all the songs for the artist
    

['"All You Want" lyrics', 'Dido Lyrics', '"All You Want"', "I'd like to watch you sleep at night", 'to hear you breathe by my side', 'And though sleep leaves me behind', "there's nowhere I'd rather be", 'And now our bed is oh so cold', 'my hands feel empty', 'no one to hold', 'I can sleep what side I want', "It's not the same with you gone", "Oh if you'd come home", "I'll let you know that", 'All you want', 'Is right here in this room', 'All you need', 'Is sitting here with you', 'All you want', "It's been three years", 'One night apart', 'but in that night you tore my heart', 'If only you had slept alone', 'If those seeds had not been sown', 'Oh you could come home and you would know that', 'All you want', 'Is right here in this room', 'All you need', 'Is sitting here with you', 'All you want', 'I hear your key turning in the door', "I won't be hearing that sound anymore", 'And you and your sin', 'can leave the way you just came in', 'send my regards to her', "I hope you've found that

In [161]:
print(f"Total run time was", (end_time - start_time))

Total run time was 0:27:20.098998


---

# Evaluation

This assignment asks you to pull data from the Twitter API and scrape www.AZLyrics.com.  After you have finished the above sections , run all the cells in this notebook. Print this to PDF and submit it, per the instructions.

In [163]:
# Simple word extractor from Peter Norvig: https://norvig.com/spell-correct.html
def words(text): 
    return re.findall(r'\w+', text.lower())

---

## Checking Twitter Data

The output from your Twitter API pull should be two files per artist, stored in files with formats like `cher_followers.txt` (a list of all follower IDs you pulled) and `cher_followers_data.txt`. These files should be in a folder named `twitter` within the repository directory. This code summarizes the information at a high level to help the instructor evaluate your work. 

In [165]:
twitter_files = os.listdir("twitter")
twitter_files = [f for f in twitter_files if f != ".DS_Store"]
artist_handles = list(set([name.split("_")[0] for name in twitter_files]))

print(f"We see two artist handles: {artist_handles[0]} and {artist_handles[1]}.")

We see two artist handles: aboveandbeyond and didoofficial.


In [180]:
for artist in artist_handles :
    follower_file = artist + "_followers.txt"
    #follower_data_file = artist + "_followers_data.txt"
    
    ids = open("twitter/" + follower_file,'r', encoding = 'utf-8').readlines()
    
    print(f"We see {len(ids)-1} in your follower file for {artist}, assuming a header row.")
    
    #with open("twitter/" + follower_data_file,'r') as infile :
    with open("twitter/" + follower_file,'r', encoding = 'utf-8') as infile :
        
        # check the headers
        headers = infile.readline().split("\t")
        
        print(f"In the follower data file ({follower_data_file}) for {artist}, we have these columns:")
        print(" : ".join(headers))
        
        description_words = []
        locations = set()
        
        
        for idx, line in enumerate(infile.readlines()) :
            line = line.strip("\n").split("\t")
            
            try : 
                locations.add(line[3])            
                description_words.extend(words(line[6]))
            except :
                pass
    
        

        print(f"We have {idx+1} data rows for {artist} in the follower data file.")

        print(f"For {artist} we have {len(locations)} unique locations.")

        print(f"For {artist} we have {len(description_words)} words in the descriptions.")
        print("Here are the five most common words:")
        print(Counter(description_words).most_common(5))

        
        print("")
        print("-"*40)
        print("")
    

We see 100912 in your follower file for aboveandbeyond, assuming a header row.
In the follower data file (aboveandbeyond_followers_data.txt) for aboveandbeyond, we have these columns:
DjSpace.booty : djspace_booty : 1568710404314124288 : Los Angeles, CA : 2022-09-10 21:18:25+00:00 : 3 : 35 : LA DJ with a passion for house music & a nice ass!

We have 100912 data rows for aboveandbeyond in the follower data file.
For aboveandbeyond we have 20389 unique locations.
For aboveandbeyond we have 100867 words in the descriptions.
Here are the five most common words:
[('74', 300), ('61', 294), ('48', 293), ('44', 291), ('55', 284)]

----------------------------------------

We see 100928 in your follower file for didoofficial, assuming a header row.
In the follower data file (aboveandbeyond_followers_data.txt) for didoofficial, we have these columns:
RenElDepre : ElRenirey : 1368982010920898561 : Santiago, Chile : 2021-03-08 17:48:50+00:00 : 10 : 195 : Lo que me Apasiona 😇 Lo que me Divierte😎 L

## Checking Lyrics 

The output from your lyrics scrape should be stored in files located in this path from the directory:
`/lyrics/[Artist Name]/[filename from URL]`. This code summarizes the information at a high level to help the instructor evaluate your work. 

In [171]:
artist_folders = os.listdir("lyrics/")
artist_folders = [f for f in artist_folders if os.path.isdir("lyrics/" + f)]

for artist in artist_folders : 
    artist_files = os.listdir("lyrics/" + artist)
    artist_files = [f for f in artist_files if 'txt' in f or 'csv' in f or 'tsv' in f]

    print(f"For {artist} we have {len(artist_files)} files.")

    artist_words = []

    for f_name in artist_files : 
        with open("lyrics/" + artist + "/" + f_name, encoding = 'utf-8') as infile : #added encoding parameter to read
            artist_words.extend(words(infile.read()))

            
    print(f"For {artist} we have roughly {len(artist_words)} words, {len(set(artist_words))} are unique.")


For abovebeyond we have 70 files.
For abovebeyond we have roughly 13104 words, 1182 are unique.
For dido we have 83 files.
For dido we have roughly 19955 words, 1537 are unique.
